In [1]:
data_bucket_name = 'classifai-backend'
dataset_name = 'test'

In [2]:
import sagemaker
from sagemaker import get_execution_role, image_uris

role = get_execution_role()
sess = sagemaker.Session()
training_image = image_uris.retrieve('image-classification', sess.boto_region_name, version='latest')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: latest.


In [20]:
# Find im2rec in our environment and set up some other vars in our environemnt

base_dir='/tmp'

%env BASE_DIR=$base_dir
%env S3_DATA_BUCKET_NAME = $data_bucket_name
%env DATASET_NAME = $dataset_name

import sys,os

# suffix='/mxnet/tools/im2rec.py'
# im2rec = list(filter( (lambda x: os.path.isfile(x + suffix )), sys.path))[0] + suffix
suffix='/incubator-mxnet/tools/im2rec.py'
im2rec = '/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/im2rec.py'
%env IM2REC=$im2rec

env: BASE_DIR=/tmp
env: S3_DATA_BUCKET_NAME=classifai-backend
env: DATASET_NAME=test
env: IM2REC=/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/im2rec.py


In [21]:
# Pull our images from S3
!aws s3 sync s3://$S3_DATA_BUCKET_NAME/public/$DATASET_NAME $BASE_DIR/$DATASET_NAME --quiet

In [22]:
%%bash
# Use the IM2REC script to convert our images into RecordIO files

# Clean up our working dir of existing LST and REC files
cd $BASE_DIR
rm *.rec
rm *.lst

# First we need to create two LST files (training and test lists), noting the correct label class for each image
# We'll also save the output of the LST files command, since it includes a list of all of our label classes
echo "Creating LST files"
python $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes

echo "Label classes:"
cat ${DATASET_NAME}_classes

# Then we create RecordIO files from the LST files
echo "Creating RecordIO files"
python $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME
python $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME
ls -lh *.rec

Creating LST files
Label classes:
Creating RecordIO files


rm: cannot remove ‘*.rec’: No such file or directory
rm: cannot remove ‘*.lst’: No such file or directory
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/im2rec.py", line 26, in <module>
    import mxnet as mx
  File "/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/../python/mxnet/__init__.py", line 23, in <module>
    from .context import Context, current_context, cpu, gpu, cpu_pinned
  File "/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/../python/mxnet/context.py", line 20, in <module>
    from .base import _LIB
  File "/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/../python/mxnet/base.py", line 312, in <module>
    _LIB = _load_lib()
  File "/home/ec2-user/SageMaker/classifai-backend/image-classification/incubator-mxnet/tools/../python/mxnet/base.py", line 279, in _load_lib
    lib_path 

CalledProcessError: Command 'b'# Use the IM2REC script to convert our images into RecordIO files\n\n# Clean up our working dir of existing LST and REC files\ncd $BASE_DIR\nrm *.rec\nrm *.lst\n\n# First we need to create two LST files (training and test lists), noting the correct label class for each image\n# We\'ll also save the output of the LST files command, since it includes a list of all of our label classes\necho "Creating LST files"\npython $IM2REC --list --recursive --pass-through --test-ratio=0.3 --train-ratio=0.7 $DATASET_NAME $DATASET_NAME > ${DATASET_NAME}_classes\n\necho "Label classes:"\ncat ${DATASET_NAME}_classes\n\n# Then we create RecordIO files from the LST files\necho "Creating RecordIO files"\npython $IM2REC --num-thread=4 ${DATASET_NAME}_train.lst $DATASET_NAME\npython $IM2REC --num-thread=4 ${DATASET_NAME}_test.lst $DATASET_NAME\nls -lh *.rec\n'' returned non-zero exit status 2.

In [23]:
sys.path

['',
 '/home/ec2-user/anaconda3/envs/python3/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/python3/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython']